<a href="https://colab.research.google.com/github/acastellanos-ie/natural_language_processing/blob/master/text_classification_practice_dl/AraBERT_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab Configuration

**Execute this steps to configure the Google Colab environment in order to execute this notebook. It is not required if you are executing it locally and you have properly configured your local environment according to what explained in the Github Repository.**

The first step is to clone the repository to have access to all the data and files.

In [ ]:
! git clone https://github.com/acastellanos-ie/natural_language_processing.git

fatal: destination path 'natural_language_processing' already exists and is not an empty directory.


Install the requirements

In [ ]:
! pip install -r natural_language_processing/arabic_requirements.txt

     |████████████████████████████████| 12.0 MB 229 kB/s 
  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-ggs943tp
  Running command git clone -q https://github.com/amaiya/eli5 /tmp/pip-req-build-ggs943tp
  Running command git checkout -b tfkeras_0_10_1 --track origin/tfkeras_0_10_1
  Switched to a new branch 'tfkeras_0_10_1'
  Branch 'tfkeras_0_10_1' set up to track remote branch 'tfkeras_0_10_1' from 'origin'.
  Cloning https://github.com/amaiya/stellargraph (to revision no_tf_dep_082) to /tmp/pip-req-build-kq9t7919
  Running command git clone -q https://github.com/amaiya/stellargraph /tmp/pip-req-build-kq9t7919
  Running command git checkout -b no_tf_dep_082 --track origin/no_tf_dep_082
  Switched to a new branch 'no_tf_dep_082'
  Branch 'no_tf_dep_082' set up to track remote branch 'no_tf_dep_082' from 'origin'.
  Using cached camel_tools-1.1.0.tar.gz (56 kB)
  Using cached stanza-1.2.2-py3-none-any.whl (337 kB)
  Using cached elasticsear

Go to the practice directory

In [ ]:
%cd natural_language_processing/text_classification_practice_dl

/content/natural_language_processing/text_classification_practice_dl


Ensure that you have the GPU runtime activated:

![](https://miro.medium.com/max/3006/1*vOkqNhJNl1204kOhqq59zA.png)

Now you have everything you need to execute the code in Colab

# HARD: Hotel Arabic-Reviews Dataset

For this practice, we focus on applying the text classification techniques covered so far to a dataset in Arabic.

In particular, we are using the [HARD: Hotel Arabic-Reviews Dataset](https://github.com/elnagara/HARD-Arabic-Dataset). This dataset contains 93700 hotel reviews in the Arabic language. The hotel reviews were collected from Booking.com website during June/July 2016. The reviews are expressed in Modern Standard Arabic as well as dialectal Arabic.
This dataset represents a common NLP challenge:  you have a set of consumer reviews that you want to analyze automatically (e.g., discover the sentiment, find the most relevant topics, ...). 

Previously, we have experimented with the original version of BERT (i.e., the one trained by Google). Although BERT has represented a considerable step ahead in the NLP research, its application is limited to English content. Since then, several initiatives have adapted the BERT architecture to contents in other languages. For this practice, we will make use of one of them: AraBERT.

In the following, we will download both the HARD dataset and the AraBERT pre-trained model.




In [ ]:
# Download the AraBERT pre-trained model
!git clone https://github.com/aub-mind/arabert

# Download and uzip the HARD dataset
!git clone https://github.com/elnagara/HARD-Arabic-Dataset
!unrar x 'HARD-Arabic-Dataset/data/unbalanced-reviews.rar'
!unzip 'HARD-Arabic-Dataset/data/balanced-reviews.zip'

fatal: destination path 'arabert' already exists and is not an empty directory.
fatal: destination path 'HARD-Arabic-Dataset' already exists and is not an empty directory.

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from HARD-Arabic-Dataset/data/unbalanced-reviews.rar

Extracting  unbalanced-reviews.txt                                         0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 

## Reading and preparing the dataset.

To enable the experimentation, we read the dataset and split the dataset into training and test



In [ ]:
!mkdir data
!mkdir train

In [ ]:
import pandas as pd
import numpy as np
import torch


from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
all_datasets= []

In [ ]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [ ]:
DATA_COLUMN = "text"
LABEL_COLUMN = "label"

In [ ]:
df_HARD = pd.read_csv("balanced-reviews.txt", sep="\t", header=0,encoding='utf-16')

df_HARD = df_HARD[["review","rating"]]  # we are interested in rating and review only
df_HARD.columns = [DATA_COLUMN, LABEL_COLUMN]
print(df_HARD[LABEL_COLUMN].value_counts())
# code rating as +ve if > 3, -ve if less, no 3s in dataset

hard_map = {
    5: 'POS',
    4: 'POS',
    2: 'NEG',
    1: 'NEG'
}

df_HARD[LABEL_COLUMN] = df_HARD[LABEL_COLUMN].apply(lambda x: hard_map[x])
train_HARD, test_HARD = train_test_split(df_HARD, test_size=0.2, random_state=42)
label_list_HARD = ['NEG', 'POS']

data_Hard = Dataset("HARD", train_HARD, test_HARD, label_list_HARD)
all_datasets.append(data_Hard)

2    38467
4    26450
5    26399
1    14382
Name: label, dtype: int64


# Modeling

Since AraBERT is not included in ktrain or other high-level libraries, we need some previous work to load the model and fine-tune it with the HARD dataset.

Luckily, HuggingFace provides a "handy" way of loading pre-trained models for many different NLP tasks. I recommend you check the HuggingFace documentation for more details on applying Deep Learning models to different scenarios and languages.

In [ ]:
!pip install pyarabic
!pip install optuna==2.3.0
!pip install transformers==4.2.1

from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import optuna 

     |████████████████████████████████| 258 kB 36.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 80 kB 9.0 MB/s 
     |████████████████████████████████| 164 kB 49.6 MB/s 
     |████████████████████████████████| 75 kB 4.8 MB/s 
     |████████████████████████████████| 111 kB 60.6 MB/s 
     |████████████████████████████████| 141 kB 59.1 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-py3-none-any.whl size=359773 sha256=781bc597c6a61ac770ca3f467ff077e1fb5c871d53e9b03a6ef50416c4c11b84
  Stored in directory: /root/.cache/pip/wheels/38/61/9e/955ab1890f6cab231b1d756db63f36c711968a324296e0b649
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=7756fa17878058300abb51953b1f13f7c8c2e48822253a0c4beefc26751242db
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69

In [ ]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [ ]:
for x in all_datasets:
  print(x.name)

HARD


You can choose which model, and dataset from here along with the max sentence length

In [ ]:
dataset_name = 'HARD'
model_name = 'aubmindlab/bert-base-arabertv02'
task_name = 'classification'
max_len = 256

The first thing to do is define a pre-processor for reading and format the data in a way that AraBERT can use.


In [ ]:
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

selected_dataset.train[DATA_COLUMN] = selected_dataset.train[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))
selected_dataset.test[DATA_COLUMN] = selected_dataset.test[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
selected_dataset.test[DATA_COLUMN]

28739     استثنائي . كل شيء . الخروج المبكراتصلو علي الس...
98861               استثنائي . . صوت الديسكو مزعج والطيارات
23662                              “ ازعاج ” . لاشي . كل شي
47107     “ لا انصح ابدا بزيارته ” . لم يعجبني شي . النظ...
3282      “ ممتعة ” . الهدوء و الخصوصية . قمت بطلب تزيين...
                                ...                        
85626     ضعيف . لاشيء . لايوجد مواقف سيارات المطعم لايت...
14421     “ عاديه واقل من عاديه ” . الافطار الاصناف قليل...
100923    ضعيف . يوجد باصات الى الحرم . - عدم نظافة الفن...
88655     “ عدم الالتزام بمنح الغرف التي يتم حجزها على ب...
59742     ضعيف . . يجب الإهتمام بالنظافة . . الأرضيات ال...
Name: text, Length: 21140, dtype: object

Now we will tokenize the data and perform some additional pre-processing steps so that AraBERT can ingest the input data.

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [ ]:
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)
train_dataset = BERTDataset(selected_dataset.train[DATA_COLUMN].to_list(),selected_dataset.train[LABEL_COLUMN].to_list(),model_name,max_len,label_map)
test_dataset = BERTDataset(selected_dataset.test[DATA_COLUMN].to_list(),selected_dataset.test[LABEL_COLUMN].to_list(),model_name,max_len,label_map)

{'NEG': 0, 'POS': 1}


OSError: ignored

The following code loads the weights of the pre-trained AraBERT model.

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In order to facilitate the model evaluation, the following function computes several classification metrics.


In [ ]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))

  macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

# Fine-tuning

In the following, we will finally fine-tune the model with the HARD dataset.

I am using some default values for the hyper-parameters. I leave as an exercise to you the hyper-parameter optimization if you want to improve the model performance.

In [ ]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8


steps_per_epoch = (len(selected_dataset.train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

2642
21136


NameError: ignored

In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()